OurCup Census Data Generation
=============================

Use this notebook to generate data on how many foreign-born residents from each country playing in the world cup live in each country in the US. This uses ACS (American Community Survey) data (05006 PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION IN THE UNITED STATES). 
1. Check for updated ACS 5-year estimate data (comes out every year), download it and change the file name below as indicated. You need to find that table and then go to "Geos" and select "Tracts -> All Tracts" to download the table we'll need here.
2. Update the "country-2-census-column.csv" file based on the teams that qualified for the World Cup

In [13]:
import pandas as pd
import json

## Figure Out Top Populations by County

In [17]:
# Load up the manual list of countries and columns in the census data they map to
# this requires making some editorial decisions
country2col_df = pd.read_csv("data/country-2-census-column.csv")

In [18]:
# Load up the ACS place of birth data for foreign-born populations
# (update this file name with the latest ACS B05006 CSV data file)
census_df = pd.read_csv('data/ACSDT5Y2022.B05006_2024-12-05T155749/ACSDT5Y2022.B05006-Data.csv', low_memory=False)

In [20]:
# find the country population for every county (this can take a few minutes)
pop_data = []
for census_idx, census_row in census_df.iterrows(): # census_df.sample(n=10).iterrows():
    if census_idx == 0:
        continue
    try:
        tract_pop_data = {}
        tract_pop_data['county'] = census_row['GEO_ID'][9:14]
        tract_pop_data['geo_id'] = census_row['GEO_ID'][9:]
        tract_pop_data['state_fips'] = census_row['GEO_ID'][9:11]
        tract_pop_data['county_fips'] = census_row['GEO_ID'][11:14]
        tract_pop_data['tract_fips'] = census_row['GEO_ID'][14:]
        for cntry_idx, cntry_row in country2col_df.iterrows():
            tract_pop_data[cntry_row['FIFA-alpha3']] = census_row[cntry_row['Census-Column-Name']]
        pop_data.append(tract_pop_data)
    except Exception as e:
        print("Failed on row {}, tract {} - {}".format(census_idx, census_row['GEO_ID'][9:], e))
pop_df = pd.DataFrame(pop_data)
pop_df.to_csv('tract-populaton-data.csv', index=False)

In [21]:
aggregated = []
for county in list(set(pop_df['county'])):
    county_data = {'fips': county}
    this_county_pop_df = pop_df[pop_df['county']==county]
    for cntry_idx, cntry_row in country2col_df.iterrows():
        county_data[cntry_row['FIFA-alpha3']] = pd.to_numeric(this_county_pop_df[cntry_row['FIFA-alpha3']]).sum()
    aggregated.append(county_data)

In [22]:
aggregated_pop_df = pd.DataFrame(aggregated)
aggregated_pop_df.to_csv('county-populaton-data.csv', index=False)

In [23]:
aggregated_pop_df[aggregated_pop_df['fips']=='13271']

,fips,ARG,AUS,BRA,CAN,CHN,COL,CRC,DEN,ENG,...,PAN,PHL,POR,IRL,ZAF,ESP,SWE,SUI,VNM,ZMB
2067,13271,17.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,8.0,70.0


## Pre-compute Rankings for each County

In [24]:
county_rankings = {}
all_ranks = []
for county in list(set(pop_df['county'])):
    this_county_ranks = []
    county_data = aggregated_pop_df[aggregated_pop_df['fips']==str(county)]
    for cntry_idx, cntry_row in country2col_df.iterrows():
        pop = county_data[cntry_row['FIFA-alpha3']]
        one_row = {'county':county, 'team':cntry_row['FIFA-alpha3'], 'pop': int(pop)}
        all_ranks.append(one_row)
        if int(pop) > 0:
            this_county_ranks.append(one_row)
    this_county_ranks.sort(key=lambda x: x['pop'], reverse=True)
    county_rankings[county] = [r['team'] for r in this_county_ranks][:3]
county_pop_df = pd.DataFrame(all_ranks)
county_pop_df.to_csv('county-populaton-data-tidy.csv', index=False)


/var/folders/v0/cxm2b8rx6z753szld5jb6k180000gp/T/ipykernel_48468/2048141002.py:8: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  one_row = {'county':county, 'team':cntry_row['FIFA-alpha3'], 'pop': int(pop)}
/var/folders/v0/cxm2b8rx6z753szld5jb6k180000gp/T/ipykernel_48468/2048141002.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(pop) > 0:


In [25]:
with open('county-recs.json', 'w') as f:
    json.dump(county_rankings, f)

Now copy and past that `county-recs.json` data into the `data/recommendations.json` file